In [ ]:
!pip install -q datasets==2.5.1

!pip install -q apache_beam==2.42.0
#mwparserfromhell

!pip install -q farm-haystack -f https://download.pytorch.org/whl/torch_stable.html

!pip install -q wikipedia==1.4.0

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
apache-beam 2.42.0 requires dill<0.3.2,>=0.3.1.1, but you have dill 0.3.5.1 which is incompatible.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
multiprocess 0.70.13 requires dill>=0.3.5.1, but you have dill 0.3.1.1 which is incompatible.


### Imports

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from pprint import pprint
import wikipedia as wiki
import re

In [ ]:
import json
from datasets import load_dataset 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Evaluation dataset

In [ ]:
# evaluation data set 1: covid qa deepset, scientific annotated data used to train deepset/roberta-base-squad2-covid
d_covid = load_dataset('covid_qa_deepset')
QA_covid = d_covid.data['train'].to_pandas()
dataQA = {'question': QA_covid['question'][:100],
          'answer':[d.get('text')[0] for d in QA_covid[:100].answers],
          'wrong_answer': [''] * 100} 
qa_dataset = pd.DataFrame(dataQA)

# evaluation data set 2: qovid qa dataset created using news platforms around the world
#qa_dataset = pd.read_csv('/content/drive/MyDrive/DeepLearning/news.csv')
#qa_dataset = qa_dataset[['question', 'answer', 'wrong_answer']]
#qa_dataset['question'] = 'Covid 19, '+ qa_dataset['question'].astype(str)
#qa_dataset.head(3)

Generating train split:   0%|          | 0/2019 [00:00<?, ? examples/s]

Dataset covid_qa_deepset downloaded and prepared to /root/.cache/huggingface/datasets/covid_qa_deepset/covid_qa_deepset/1.0.0/fb886523842e312176f92ec8e01e77a08fa15a694f5741af6fc42796ee9c8c46. Subsequent calls will reuse this data.


  0%|          | 0/1 [00:00<?, ?it/s]

In [ ]:
qa_dataset

,question,answer,wrong_answer
0,What is the main cause of HIV-1 infection in c...,Mother-to-child transmission (MTCT) is the mai...,
1,What plays the crucial role in the Mother to C...,DC-SIGNR plays a crucial role in MTCT of HIV-1...,
2,How many children were infected by HIV-1 in 20...,"more than 400,000 children were infected world...",
3,What is the role of C-C Motif Chemokine Ligand...,"High copy numbers of CCL3L1, a potent HIV-1 su...",
4,What is DC-GENR and where is it expressed?,Dendritic cell-specific ICAM-grabbing non-inte...,
...,...,...,...
95,How is exhaled breath condensate used in viral...,the isolation of respiratory viruses,
96,How many patients were i this study?,102,
97,What was the conclusion of this study?,EBC collection using the RTube™ is not reliabl...,
98,How long did the patient breath into the RTube?,10 minutes,


### Elasticsearch Server

```bash
docker pull docker.elastic.co/elasticsearch/elasticsearch:7.9.2

docker run -d -p 9200:9200 -e "discovery.type=single-node" elasticsearch:7.9.2

```

In [ ]:
%%bash

wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
chown -R daemon:daemon elasticsearch-7.9.2

In [ ]:
%%bash --bg

sudo -u daemon -- elasticsearch-7.9.2/bin/elasticsearch

In [ ]:
import time
time.sleep(30)

### Reader imports

In [ ]:
# just to display the different retrievers available
from haystack.nodes import (
    BM25Retriever,
    TfidfRetriever,
    DensePassageRetriever,
    FARMReader,
    RAGenerator,
    BaseComponent,
    JoinDocuments,
)

from haystack.pipelines import (
    ExtractiveQAPipeline, 
    DocumentSearchPipeline, 
    GenerativeQAPipeline
)

from haystack.nodes import FARMReader

from haystack.utils import print_answers

# Definition: MODEL, GPU, TOP_K, READER

In [ ]:
# CHANGE MODEL NAME FOR DIFFERENT MODELs
# squad 2 model
model = "deepset/roberta-base-squad2"

# squad 2 model trained on covid 19 data
# model = "deepset/roberta-base-squad2-covid"


use_gpu = True

top_k_retriever = 10
top_k_reader = 5


reader = FARMReader(
    model_name_or_path=model, #"deepset/roberta-base-squad2"
    use_gpu=use_gpu
)

# Covid 

### Context: COVID-19 Dataset [Link here](https://github.com/deepset-ai/COVID-QA)

In [ ]:
with open('/content/drive/MyDrive/DeepLearning/COVID-QA.json') as jsonfile:
    data = json.load(jsonfile)

# create the dataframe that holds our data
df_covid = pd.DataFrame()
for i in range(len(data['data'])):
    q = pd.json_normalize(data['data'][i]['paragraphs'][0]['qas'])
    contxt = data['data'][i]['paragraphs'][0]['context']
    d_id = data['data'][i]['paragraphs'][0]['document_id']
    q['context'] = contxt
    q['document_id'] = d_id
    df_covid = pd.concat([df_covid, q])
    
df_covid.reset_index(drop=True, inplace=True)
df_covid.head(3)

,question,id,answers,is_impossible,context,document_id
0,What is the main cause of HIV-1 infection in children?,262,[{'text': 'Mother-to-child transmission (MTCT) is the main cause of HIV-1 in...,False,Functional Genetic Variants in DC-SIGNR Are Associated with Mother-to-Child ...,630
1,What plays the crucial role in the Mother to Child Transmission of HIV-1 and...,276,[{'text': 'DC-SIGNR plays a crucial role in MTCT of HIV-1 and that impaired ...,False,Functional Genetic Variants in DC-SIGNR Are Associated with Mother-to-Child ...,630
2,"How many children were infected by HIV-1 in 2008-2009, worldwide?",278,"[{'text': 'more than 400,000 children were infected worldwide, mostly throug...",False,Functional Genetic Variants in DC-SIGNR Are Associated with Mother-to-Child ...,630


In [ ]:
df_covid['context_cleaned'] = df_covid.context.apply(
    lambda x: re.sub(r'https?:\/\/.*?[\s+]|\n|[^a-zA-z0-9.]', ' ', x)
)

### Document Store

In [ ]:
%%bash

wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
chown -R daemon:daemon elasticsearch-7.9.2

In [ ]:
%%bash --bg

sudo -u daemon -- elasticsearch-7.9.2/bin/elasticsearch

In [ ]:
import time
time.sleep(30)

In [ ]:
documents_lst_covid = df_covid.to_dict(orient='records')

In [ ]:
# the way document_store wants data formatted
dicts_covid = [
    {
        'content' : str(elm['context']),
        'meta' : {
            'name' : str(elm['question'])
        }
    } 
    for elm in tqdm(documents_lst_covid)
]

100%|██████████| 2019/2019 [00:00<00:00, 441563.24it/s]


Write to document store

In [ ]:
%%time
from haystack.document_stores import ElasticsearchDocumentStore
document_store_covid = ElasticsearchDocumentStore(
    port=9200
)
document_store_covid.delete_documents()
document_store_covid.write_documents(
    documents=dicts_covid
)

CPU times: user 832 ms, sys: 103 ms, total: 935 ms
Wall time: 12.4 s


### Retrieving answers

#### BM25

In [ ]:
retriever = BM25Retriever(document_store=document_store_covid)

reader = FARMReader(
    model_name_or_path=model, #"deepset/roberta-base-squad2"
    use_gpu=use_gpu
)

pipe_covid = ExtractiveQAPipeline(reader, retriever)

df_res_covid_bm25  = qa_dataset.copy()

# prepare columns for answers
df_res_covid_bm25['predictions_covid_context_bm25'] = [list() for x in range(len(df_res_covid_bm25.index))]

for q_i in tqdm(range(len(qa_dataset.question.tolist()))):

    print('question : ', qa_dataset.question[q_i])

    try:
        # covid dataset context prediction
        prediction_covid = pipe_covid.run(
            query=qa_dataset.question[q_i],
            params={
                "Retriever" : {"top_k": top_k_retriever},
                "Reader": {"top_k": top_k_reader}
            }
        )

        df_res_covid_bm25.loc[q_i, 'predictions_covid_context_bm25'].append(
            [prediction_covid['answers'][k].answer for k in range(len(prediction_covid['answers']))]
        )
    except:
        df_res_covid_bm25.loc[q_i, 'predictions_covid_context_bm25'].append([])

    
df_res_covid_bm25.to_csv('/content/drive/MyDrive/DeepLearning/df_res_covid_bm25_org_data.csv', index=False)

  0%|          | 0/100 [00:00<?, ?it/s]

question :  What is the main cause of HIV-1 infection in children?



Inferencing Samples: 100%|██████████| 11/11 [00:07<00:00,  1.38 Batches/s]
ERROR:haystack.modeling.model.predictions:Invalid end offset: 
(-26973, -26956) with a span answer. 
ERROR:haystack.modeling.model.predictions:Invalid end offset: 
(-19766, -19714) with a span answer. 
  1%|          | 1/100 [00:08<13:47,  8.36s/it]

question :  What plays the crucial role in the Mother to Child Transmission of HIV-1 and what increases the risk



Inferencing Samples: 100%|██████████| 11/11 [00:07<00:00,  1.50 Batches/s]
ERROR:haystack.modeling.model.predictions:Invalid end offset: 
(-31976, -31966) with a span answer. 
ERROR:haystack.modeling.model.predictions:Invalid end offset: 
(-21838, -21822) with a span answer. 
ERROR:haystack.modeling.model.predictions:Invalid end offset: 
(-32255, -32245) with a span answer. 
ERROR:haystack.modeling.model.predictions:Invalid end offset: 
(-31706, -31676) with a span answer. 
ERROR:haystack.modeling.model.predictions:Invalid end offset: 
(-31274, -31241) with a span answer. 
ERROR:haystack.modeling.model.predictions:Invalid end offset: 
(-30113, -30094) with a span answer. 
  2%|▏         | 2/100 [00:16<13:33,  8.30s/it]

...


Inferencing Samples: 100%|██████████| 12/12 [00:09<00:00,  1.32 Batches/s]
ERROR:haystack.modeling.model.predictions:Invalid end offset: 
(-21412, -21360) with a span answer. 
ERROR:haystack.modeling.model.predictions:Invalid end offset: 
(-21175, -21111) with a span answer. 
ERROR:haystack.modeling.model.predictions:Invalid end offset: 
(-17016, -16973) with a span answer. 
ERROR:haystack.modeling.model.predictions:Invalid end offset: 
(-26287, -26252) with a span answer. 
ERROR:haystack.modeling.model.predictions:Invalid end offset: 
(-6170, -6115) with a span answer. 
 98%|█████████▊| 98/100 [15:08<00:17,  8.67s/it]

question :  How long did the patient breath into the RTube?



Inferencing Samples: 100%|██████████| 12/12 [00:08<00:00,  1.37 Batches/s]
ERROR:haystack.modeling.model.predictions:Invalid end offset: 
(-30511, -30483) with a span answer. 
ERROR:haystack.modeling.model.predictions:Invalid end offset: 
(-29592, -29585) with a span answer. 
ERROR:haystack.modeling.model.predictions:Invalid end offset: 
(-23707, -23693) with a span answer. 
ERROR:haystack.modeling.model.predictions:Invalid end offset: 
(-25044, -25035) with a span answer. 
ERROR:haystack.modeling.model.predictions:Invalid end offset: 
(-9540, -9536) with a span answer. 
ERROR:haystack.modeling.model.predictions:Invalid end offset: 
(-430, -428) with a span answer. 
 99%|█████████▉| 99/100 [15:17<00:08,  8.79s/it]

question :  What followed the reverse transcription step in the analysis?



Inferencing Samples: 100%|██████████| 12/12 [00:09<00:00,  1.31 Batches/s]
ERROR:haystack.modeling.model.predictions:Invalid end offset: 
(-17683, -17657) with a span answer. 
ERROR:haystack.modeling.model.predictions:Invalid end offset: 
(-21286, -21254) with a span answer. 
ERROR:haystack.modeling.model.predictions:Invalid end offset: 
(-25040, -25024) with a span answer. 
100%|██████████| 100/100 [15:26<00:00,  9.27s/it]


#### RE-INITIALISE ELASTICSEARCH

In [ ]:
%%bash

wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
chown -R daemon:daemon elasticsearch-7.9.2

In [ ]:
%%bash --bg

sudo -u daemon -- elasticsearch-7.9.2/bin/elasticsearch

In [ ]:
import time
time.sleep(30)

Write to document store

In [ ]:
%%time
from haystack.document_stores import ElasticsearchDocumentStore
document_store_covid = ElasticsearchDocumentStore(
    port=9200
)
document_store_covid.delete_documents()
document_store_covid.write_documents(
    documents=dicts_covid
)

CPU times: user 822 ms, sys: 87.4 ms, total: 909 ms
Wall time: 11.4 s


#### TF IDF

In [ ]:
retriever_tfidf = TfidfRetriever(document_store=document_store_covid)

reader = FARMReader(
    model_name_or_path=model, #"deepset/roberta-base-squad2"
    use_gpu=use_gpu
)

pipe_covid_tfidf = ExtractiveQAPipeline(reader, retriever_tfidf)

df_res_covid_tfidf = qa_dataset.copy()

# prepare columns for answers
df_res_covid_tfidf['predictions_covid_context_tfidf'] = [list() for x in range(len(df_res_covid_tfidf.index))]

for q_i in tqdm(range(len(qa_dataset.question.tolist()))):

    print('question : ', qa_dataset.question[q_i])

    try:
        # covid dataset context prediction
        prediction_covid = pipe_covid_tfidf.run(
            query=qa_dataset.question[q_i],
            params={
                "Retriever" : {"top_k": top_k_retriever},
                "Reader": {"top_k": top_k_reader}
            }
        )

        df_res_covid_tfidf.loc[q_i, 'predictions_covid_context_tfidf'].append(
            [prediction_covid['answers'][k].answer for k in range(len(prediction_covid['answers']))]
        )
    except:
        df_res_covid_tfidf.loc[q_i, 'predictions_covid_context_tfidf'].append([])


df_res_covid_tfidf.to_csv('/content/drive/MyDrive/DeepLearning/df_res_covid_tfidf_org_data.csv', index=False)

  0%|          | 0/100 [00:00<?, ?it/s]

question :  What is the main cause of HIV-1 infection in children?



  1%|          | 1/100 [00:00<00:35,  2.82it/s]

question :  What plays the crucial role in the Mother to Child Transmission of HIV-1 and what increases the risk



  2%|▏         | 2/100 [00:00<00:29,  3.30it/s]

...


 98%|█████████▊| 98/100 [00:28<00:00,  4.31it/s]

question :  How long did the patient breath into the RTube?



 99%|█████████▉| 99/100 [00:28<00:00,  4.30it/s]

question :  What followed the reverse transcription step in the analysis?



100%|██████████| 100/100 [00:29<00:00,  3.42it/s]


#### RE-INITIALISE ELASTICSEARCH

In [ ]:
%%bash

wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
chown -R daemon:daemon elasticsearch-7.9.2

In [ ]:
%%bash --bg

sudo -u daemon -- elasticsearch-7.9.2/bin/elasticsearch

In [ ]:
import time
time.sleep(30)

In [ ]:
%%time
from haystack.document_stores import ElasticsearchDocumentStore
document_store_covid = ElasticsearchDocumentStore(
    port=9200
)
document_store_covid.delete_documents()
document_store_covid.write_documents(
    documents=dicts_covid
)

CPU times: user 825 ms, sys: 117 ms, total: 942 ms
Wall time: 11.6 s


#### DensePassageRetriever

In [ ]:
from haystack.nodes import DensePassageRetriever  
retriever_DPR = DensePassageRetriever(document_store=document_store_covid,
    query_embedding_model="bert-base-uncased",
    passage_embedding_model="bert-base-uncased"
)

document_store_covid.update_embeddings(retriever_DPR)

reader = FARMReader(
    model_name_or_path=model, #"deepset/roberta-base-squad2"
    use_gpu=use_gpu
)

pipe_covid_DPR = ExtractiveQAPipeline(reader, retriever_tfidf)

df_res_covid_DPR = qa_dataset.copy()

# prepare columns for answers
df_res_covid_DPR['predictions_covid_context_DPR'] = [list() for x in range(len(df_res_covid_DPR.index))]

for q_i in tqdm(range(len(qa_dataset.question.tolist()))):

    print('question : ', qa_dataset.question[q_i])

    try:
        # covid dataset context prediction
        prediction_covid = pipe_covid_DPR.run(
            query=qa_dataset.question[q_i],
            params={
                "Retriever" : {"top_k": top_k_retriever},
                "Reader": {"top_k": top_k_reader}
            }
        )

        df_res_covid_DPR.loc[q_i, 'predictions_covid_context_DPR'].append(
            [prediction_covid['answers'][k].answer for k in range(len(prediction_covid['answers']))]
        )
    except:
        df_res_covid_DPR.loc[q_i, 'predictions_covid_context_DPR'].append([])
    

df_res_covid_DPR.to_csv('/content/drive/MyDrive/DeepLearning/df_res_covid_DPR_org_data.csv', index=False)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizerFast'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizerFast'.


Updating embeddings:   0%|          | 0/147 [00:00<?, ? Docs/s]

Create embeddings:   0%|          | 0/160 [00:00<?, ? Docs/s]

  0%|          | 0/100 [00:00<?, ?it/s]

question :  What is the main cause of HIV-1 infection in children?


...


 98%|█████████▊| 98/100 [00:28<00:00,  4.32it/s]

question :  How long did the patient breath into the RTube?



 99%|█████████▉| 99/100 [00:28<00:00,  4.28it/s]

question :  What followed the reverse transcription step in the analysis?



100%|██████████| 100/100 [00:29<00:00,  3.43it/s]


# Wikipedia

In [ ]:
## if not already downloaded
# ds = load_dataset('wikipedia', "20220301.simple")
## We take the training data and convert it to a Pandas DataFrame
# df = ds.data['train'].to_pandas()

In [ ]:
## if not already downloaded
ds = load_dataset('wikipedia', "20220301.simple")
## We take the training data and convert it to a Pandas DataFrame
df_wiki = ds.data['train'].to_pandas()
df_wiki.head(3)

  0%|          | 0/1 [00:00<?, ?it/s]

,id,url,title,text
0,1,https://simple.wikipedia.org/wiki/April,April,"April is the fourth month of the year in the Julian and Gregorian calendars,..."
1,2,https://simple.wikipedia.org/wiki/August,August,"August (Aug.) is the eighth month of the year in the Gregorian calendar, com..."
2,6,https://simple.wikipedia.org/wiki/Art,Art,Art is a creative activity that expresses imaginative or technical skill. It...


In [ ]:
import re
df_wiki['context_cleaned'] = df_wiki.text.apply(
    lambda x: re.sub(r'https?:\/\/.*?[\s+]|\n|[^a-zA-z0-9.]', ' ', x)
)

### Document Store

In [ ]:
%%bash

wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
chown -R daemon:daemon elasticsearch-7.9.2

In [ ]:
%%bash --bg

sudo -u daemon -- elasticsearch-7.9.2/bin/elasticsearch

In [ ]:
import time
time.sleep(30)

In [ ]:
documents_lst_wiki = df_wiki.to_dict(orient='records')

In [ ]:
# the way document_store wants data formatted
dicts_wiki = [
    {
        'content' : elm['text'],
        'meta' : {
            'name' : elm['title']
        }
    } 
    for elm in tqdm(documents_lst_wiki)
]

100%|██████████| 205328/205328 [00:00<00:00, 363988.95it/s]


In [ ]:
%%time
document_store_wiki = ElasticsearchDocumentStore(
    port=9200
)
document_store_wiki.delete_documents()
document_store_wiki.write_documents(
    documents=dicts_wiki
)

CPU times: user 16.4 s, sys: 3.33 s, total: 19.7 s
Wall time: 7min 12s


### Retrieving answers

#### BM25

In [ ]:
retriever = BM25Retriever(document_store=document_store_wiki)

reader = FARMReader(
    model_name_or_path=model, #"deepset/roberta-base-squad2"
    use_gpu=use_gpu
)

pipe_wiki = ExtractiveQAPipeline(reader, retriever)

df_res_wiki_bm25 = qa_dataset.copy()

# prepare columns for answers
df_res_wiki_bm25['predictions_wiki_context_bm25'] = [list() for x in range(len(df_res_wiki_bm25.index))]

for q_i in tqdm(range(len(qa_dataset.question.tolist()))):
    
    print('question : ', qa_dataset.question[q_i])
    
    try:
        # covid dataset context prediction
        prediction_wiki = pipe_wiki.run(
            query=qa_dataset.question[q_i],
            params={
                "Retriever" : {"top_k": top_k_retriever},
                "Reader": {"top_k": top_k_reader}
            }
        )

        df_res_wiki_bm25.loc[q_i, 'predictions_wiki_context_bm25'].append(
            [prediction_wiki['answers'][k].answer for k in range(len(prediction_wiki['answers']))]
        )
    except:
        df_res_wiki_bm25.loc[q_i, 'predictions_wiki_context_bm25'].append([])

df_res_wiki_bm25.to_csv('/content/drive/MyDrive/DeepLearning/df_res_wiki_bm25_org_data.csv', index=False)

  0%|          | 0/100 [00:00<?, ?it/s]

question :  What is the main cause of HIV-1 infection in children?



  1%|          | 1/100 [00:00<01:23,  1.18it/s]

question :  What plays the crucial role in the Mother to Child Transmission of HIV-1 and what increases the risk



  2%|▏         | 2/100 [00:02<01:44,  1.06s/it]

...


 98%|█████████▊| 98/100 [01:05<00:01,  1.25it/s]

question :  How long did the patient breath into the RTube?



 99%|█████████▉| 99/100 [01:06<00:00,  1.16it/s]

question :  What followed the reverse transcription step in the analysis?



100%|██████████| 100/100 [01:06<00:00,  1.50it/s]


#### RE-INITIALISE ELASTICSEARCH

In [ ]:
%%bash

wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
chown -R daemon:daemon elasticsearch-7.9.2

In [ ]:
%%bash --bg

sudo -u daemon -- elasticsearch-7.9.2/bin/elasticsearch

In [ ]:
import time
time.sleep(30)

In [ ]:
documents_lst_wiki = df_wiki.to_dict(orient='records')

In [ ]:
# the way document_store wants data formatted
dicts_wiki = [
    {
        'content' : elm['text'],
        'meta' : {
            'name' : elm['title']
        }
    } 
    for elm in tqdm(documents_lst_wiki)
]

100%|██████████| 205328/205328 [00:00<00:00, 310810.17it/s]


In [ ]:
%%time
document_store_wiki = ElasticsearchDocumentStore(
    port=9200
)
document_store_wiki.delete_documents()
document_store_wiki.write_documents(
    documents=dicts_wiki
)

CPU times: user 16.6 s, sys: 899 ms, total: 17.5 s
Wall time: 7min 29s


#### TF IDF

In [ ]:
retriever = TfidfRetriever(document_store=document_store_wiki)

reader = FARMReader(
    model_name_or_path=model, #"deepset/roberta-base-squad2"
    use_gpu=use_gpu
)

pipe_wiki = ExtractiveQAPipeline(reader, retriever)

df_res_wiki_tfidf = qa_dataset.copy()

# prepare columns for answers
df_res_wiki_tfidf['predictions_wiki_context_tfidf'] = [list() for x in range(len(df_res_wiki_tfidf.index))]

for q_i in tqdm(range(len(qa_dataset.question.tolist()))):
    
    print('question : ', qa_dataset.question[q_i])
    
    try:
        # covid dataset context prediction
        prediction_wiki = pipe_wiki.run(
            query=qa_dataset.question[q_i],
            params={
                "Retriever" : {"top_k": top_k_retriever},
                "Reader": {"top_k": top_k_reader}
            }
        )

        df_res_wiki_tfidf.loc[q_i, 'predictions_wiki_context_tfidf'].append(
            [prediction_wiki['answers'][k].answer for k in range(len(prediction_wiki['answers']))]
        )
    except:
        df_res_wiki_tfidf.loc[q_i, 'predictions_wiki_context_tfidf'].append([])


df_res_wiki_tfidf.to_csv('/content/drive/MyDrive/DeepLearning/df_res_wiki_tfidf_org_data.csv', index=False)

  0%|          | 0/100 [00:00<?, ?it/s]

question :  What is the main cause of HIV-1 infection in children?



  1%|          | 1/100 [00:03<04:58,  3.02s/it]

question :  What plays the crucial role in the Mother to Child Transmission of HIV-1 and what increases the risk



  2%|▏         | 2/100 [00:05<04:46,  2.92s/it]

...


 98%|█████████▊| 98/100 [04:12<00:04,  2.43s/it]

question :  How long did the patient breath into the RTube?



 99%|█████████▉| 99/100 [04:14<00:02,  2.47s/it]

question :  What followed the reverse transcription step in the analysis?



100%|██████████| 100/100 [04:17<00:00,  2.58s/it]


#### RE-INITIALISE ELASTICSEARCH

In [ ]:
%%bash

wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
chown -R daemon:daemon elasticsearch-7.9.2

In [ ]:
%%bash --bg

sudo -u daemon -- elasticsearch-7.9.2/bin/elasticsearch

In [ ]:
import time
time.sleep(30)

In [ ]:
documents_lst_wiki = df_wiki.to_dict(orient='records')

In [ ]:
# the way document_store wants data formatted
dicts_wiki = [
    {
        'content' : elm['text'],
        'meta' : {
            'name' : elm['title']
        }
    } 
    for elm in tqdm(documents_lst_wiki)
]

100%|██████████| 205328/205328 [00:00<00:00, 274181.01it/s]


In [ ]:
%%time
from haystack.document_stores import ElasticsearchDocumentStore
document_store_wiki = ElasticsearchDocumentStore(
    port=9200
)
document_store_wiki.delete_documents()
document_store_wiki.write_documents(
    documents=dicts_wiki
)

CPU times: user 16.6 s, sys: 530 ms, total: 17.2 s
Wall time: 7min 10s


#### DensePassageRetriever

In [ ]:
from haystack.nodes import DensePassageRetriever  
retriever_DPR = DensePassageRetriever(document_store=document_store_wiki,
    query_embedding_model="bert-base-uncased",
    passage_embedding_model="bert-base-uncased"
)

document_store_wiki.update_embeddings(retriever_DPR)

reader = FARMReader(
    model_name_or_path=model, #"deepset/roberta-base-squad2"
    use_gpu=use_gpu
)

pipe_covid_DPR = ExtractiveQAPipeline(reader, retriever_tfidf)

df_res_wiki_DPR = qa_dataset.copy()

# prepare columns for answers
df_res_wiki_DPR['predictions_wiki_context_DPR'] = [list() for x in range(len(df_res_wiki_DPR.index))]

for q_i in tqdm(range(len(qa_dataset.question.tolist()))):

    print('question : ', qa_dataset.question[q_i])

    try:
        # covid dataset context prediction
        prediction_covid = pipe_covid_DPR.run(
            query=qa_dataset.question[q_i],
            params={
                "Retriever" : {"top_k": top_k_retriever},
                "Reader": {"top_k": top_k_reader}
            }
        )

        df_res_wiki_DPR.loc[q_i, 'predictions_wiki_context_DPR'].append(
            [prediction_covid['answers'][k].answer for k in range(len(prediction_covid['answers']))]
        )
    except:
        df_res_wiki_DPR.loc[q_i, 'predictions_wiki_context_DPR'].append([])

df_res_wiki_DPR.to_csv('/content/drive/MyDrive/DeepLearning/df_res_wiki_DPR.csv', index=False)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'DPRQuestionEncoderTokenizerFast'.
The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertTokenizer'. 
The class this function is called from is 'DPRContextEncoderTokenizerFast'.


Updating embeddings:   0%|          | 0/205308 [00:00<?, ? Docs/s]

Create embeddings:   0%|          | 0/10000 [00:00<?, ? Docs/s]

Create embeddings:   0%|          | 0/10000 [00:00<?, ? Docs/s]

Create embeddings:   0%|          | 0/10000 [00:00<?, ? Docs/s]

...


 98%|█████████▊| 98/100 [06:52<00:05,  2.59s/it]

question :  How long did the patient breath into the RTube?



 99%|█████████▉| 99/100 [06:55<00:02,  2.64s/it]

question :  What followed the reverse transcription step in the analysis?



100%|██████████| 100/100 [06:58<00:00,  4.18s/it]


# Wikipedia API

### Document store

In [ ]:
%%bash

wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
chown -R daemon:daemon elasticsearch-7.9.2

In [ ]:
%%bash --bg

sudo -u daemon -- elasticsearch-7.9.2/bin/elasticsearch

In [ ]:
import time
time.sleep(30)

In [ ]:
%%time
from haystack.document_stores import ElasticsearchDocumentStore

document_store_wikiAPI = ElasticsearchDocumentStore(
    port=9200
)
document_store_wikiAPI.delete_documents()

CPU times: user 18 ms, sys: 6.98 ms, total: 25 ms
Wall time: 2.2 s


### Retrieving answers

#### BM25

In [ ]:
reader = FARMReader(
    model_name_or_path=model, #"deepset/roberta-base-squad2"
    use_gpu=use_gpu
)

df_res_wikiAPI_bm25 = qa_dataset.copy()

# prepare columns for answers
df_res_wikiAPI_bm25['predictions_wikiAPI_context_bm25'] = [list() for x in range(len(df_res_wikiAPI_bm25.index))]


for q_i in tqdm(range(len(qa_dataset.question.tolist()))):
    
    print('question : ', qa_dataset.question[q_i])
    
    # wikipedia API context prediction 
    try:
        search_results_wikiAPI = wiki.search(qa_dataset.question[q_i])

        wiki_pages = [
            wiki.page(res)
            for res in search_results_wikiAPI[:top_k_reader]
        ]
        
        dicts_wikiAPI = [
            {
                'content' : wiki_page.content,
                'meta' : {
                    'name' : wiki_page.title
                }
            } 
            for wiki_page in tqdm(wiki_pages)
        ]
        
        
        document_store_wikiAPI.delete_documents()
        document_store_wikiAPI.write_documents(dicts_wikiAPI)
        
        retriever_wikiAPI = BM25Retriever(document_store=document_store_wikiAPI)
        
        pipe_wikiAPI = ExtractiveQAPipeline(reader, retriever_wikiAPI)
        
        prediction_wikiAPI = pipe_wikiAPI.run(
            query=qa_dataset.question[q_i],
            params={
                "Retriever" : {"top_k": top_k_retriever},
                "Reader": {"top_k": top_k_reader}
            }
        )

        df_res_wikiAPI_bm25.loc[q_i, 'predictions_wikiAPI_context_bm25'].append(
            [prediction_wikiAPI['answers'][k].answer for k in range(len(prediction_wikiAPI['answers']))]
        )
    except:
        df_res_wikiAPI_bm25.loc[q_i, 'predictions_wikiAPI_context_bm25'].append([])


df_res_wikiAPI_bm25.to_csv('/content/drive/MyDrive/DeepLearning/df_res_wikiAPI_bm25_org_data.csv', index=False)

  0%|          | 0/100 [00:00<?, ?it/s]

question :  What is the main cause of HIV-1 infection in children?


/usr/local/lib/python3.7/dist-packages/wikipedia/wikipedia.py:389: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.7/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')
  1%|          | 1/100 [00:01<02:45,  1.67s/it]

question :  What plays the crucial role in the Mother to Child Transmission of HIV-1 and what increases the risk


  2%|▏         | 2/100 [00:03<02:51,  1.75s/it]

...

 98%|█████████▊| 98/100 [08:39<00:08,  4.27s/it]

question :  How long did the patient breath into the RTube?



0it [00:00, ?it/s]
 99%|█████████▉| 99/100 [08:42<00:03,  3.67s/it]

question :  What followed the reverse transcription step in the analysis?



100%|██████████| 5/5 [00:01<00:00,  3.73it/s]

Inferencing Samples: 100%|██████████| 5/5 [00:03<00:00,  1.49 Batches/s]
ERROR:haystack.modeling.model.predictions:Invalid end offset: 
(-22461, -22450) with a span answer. 
100%|██████████| 100/100 [08:51<00:00,  5.32s/it]


#### RE-INITIALISE ELASTICSEARCH

In [ ]:
%%bash

wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
chown -R daemon:daemon elasticsearch-7.9.2

In [ ]:
%%bash --bg

sudo -u daemon -- elasticsearch-7.9.2/bin/elasticsearch

In [ ]:
import time
time.sleep(30)

In [ ]:
document_store_wikiAPI = ElasticsearchDocumentStore(
    port=9200
)
document_store_wikiAPI.delete_documents()

#### TF IDF

In [ ]:
reader = FARMReader(
    model_name_or_path=model, #"deepset/roberta-base-squad2"
    use_gpu=use_gpu
)

df_res_wikiAPI_tfidf = qa_dataset.copy()

# prepare columns for answers
df_res_wikiAPI_tfidf['predictions_wikiAPI_context_tfidf'] = [list() for x in range(len(df_res_wikiAPI_tfidf.index))]


for q_i in tqdm(range(len(qa_dataset.question.tolist()))):
    
    print('question : ', qa_dataset.question[q_i])
    
    try:
        # wikipedia API context prediction 
        search_results_wikiAPI = wiki.search(qa_dataset.question[q_i])

        wiki_pages = [
            wiki.page(res)
            for res in search_results_wikiAPI[:top_k_reader]
        ]
        
        dicts_wikiAPI = [
            {
                'content' : wiki_page.content,
                'meta' : {
                    'name' : wiki_page.title
                }
            } 
            for wiki_page in tqdm(wiki_pages)
        ]
        
        
        document_store_wikiAPI.delete_documents()
        document_store_wikiAPI.write_documents(dicts_wikiAPI)
        
        retriever_wikiAPI = TfidfRetriever(document_store=document_store_wikiAPI)
        
        pipe_wikiAPI = ExtractiveQAPipeline(reader, retriever_wikiAPI)
        
        prediction_wikiAPI = pipe_wikiAPI.run(
            query=qa_dataset.question[q_i],
            params={
                "Retriever" : {"top_k": top_k_retriever},
                "Reader": {"top_k": top_k_reader}
            }
        )

        df_res_wikiAPI_tfidf.loc[q_i, 'predictions_wikiAPI_context_tfidf'].append(
            [prediction_wikiAPI['answers'][k].answer for k in range(len(prediction_wikiAPI['answers']))]
        )
    except:
        df_res_wikiAPI_tfidf.loc[q_i, 'predictions_wikiAPI_context_tfidf'].append([])

df_res_wikiAPI_tfidf.to_csv('/content/drive/MyDrive/DeepLearning/df_res_wikiAPI_tfidf_org_data.csv', index=False)

  0%|          | 0/100 [00:00<?, ?it/s]

question :  What is the main cause of HIV-1 infection in children?


/usr/local/lib/python3.7/dist-packages/wikipedia/wikipedia.py:389: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.7/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')
  1%|          | 1/100 [00:00<01:23,  1.19it/s]

question :  What plays the crucial role in the Mother to Child Transmission of HIV-1 and what increases the risk


  3%|▎         | 3/100 [00:01<00:39,  2.43it/s]

...

/usr/local/lib/python3.7/dist-packages/wikipedia/wikipedia.py:389: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file /usr/local/lib/python3.7/dist-packages/wikipedia/wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')
 98%|█████████▊| 98/100 [04:22<00:03,  1.82s/it]

question :  How long did the patient breath into the RTube?



0it [00:00, ?it/s]
 99%|█████████▉| 99/100 [04:24<00:01,  1.88s/it]

question :  What followed the reverse transcription step in the analysis?



100%|██████████| 5/5 [00:01<00:00,  3.89it/s]

100%|██████████| 100/100 [04:30<00:00,  2.70s/it]


#### RE-INITIALISE ELASTICSEARCH

In [ ]:
%%bash

wget https://artifacts.elastic.co/downloads/elasticsearch/elasticsearch-7.9.2-linux-x86_64.tar.gz -q
tar -xzf elasticsearch-7.9.2-linux-x86_64.tar.gz
chown -R daemon:daemon elasticsearch-7.9.2

In [ ]:
%%bash --bg

sudo -u daemon -- elasticsearch-7.9.2/bin/elasticsearch

In [ ]:
import time
time.sleep(30)

In [ ]:
%%time
from haystack.document_stores import ElasticsearchDocumentStore
document_store_wikiAPI = ElasticsearchDocumentStore(
    port=9200
)
document_store_wikiAPI.delete_documents()

CPU times: user 29.7 ms, sys: 3.17 ms, total: 32.9 ms
Wall time: 2.04 s


# Combining dataframes 

In [ ]:
df_res_covid_tfidf = pd.read_csv('/content/drive/MyDrive/DeepLearning/df_res_covid_tfidf_org_data.csv')
df_res_covid_bm25 = pd.read_csv('/content/drive/MyDrive/DeepLearning/df_res_covid_bm25_org_data.csv')
df_res_wiki_bm25 = pd.read_csv('/content/drive/MyDrive/DeepLearning/df_res_wiki_bm25_org_data.csv')
df_res_wiki_tfidf = pd.read_csv('/content/drive/MyDrive/DeepLearning/df_res_wiki_tfidf_org_data.csv')
df_res_wikiAPI_bm25 = pd.read_csv('/content/drive/MyDrive/DeepLearning/df_res_wikiAPI_bm25_org_data.csv')
df_res_wikiAPI_tfidf = pd.read_csv('/content/drive/MyDrive/DeepLearning/df_res_wikiAPI_tfidf_org_data.csv')

In [ ]:
df_res = qa_dataset.copy()

In [ ]:
df_res['predictions_covid_context_tfidf'] = df_res_covid_tfidf['predictions_covid_context_tfidf'].copy()
df_res['predictions_covid_context_bm25'] = df_res_covid_bm25['predictions_covid_context_bm25'].copy()
df_res['predictions_wiki_context_bm25'] = df_res_wiki_bm25['predictions_wiki_context_bm25'].copy()
df_res['predictions_wiki_context_tfidf'] = df_res_wiki_tfidf['predictions_wiki_context_tfidf'].copy()
df_res['predictions_wikiAPI_context_bm25'] = df_res_wikiAPI_bm25['predictions_wikiAPI_context_bm25'].copy()
df_res['predictions_wikiAPI_context_tfidf'] = df_res_wikiAPI_tfidf['predictions_wikiAPI_context_tfidf'].copy()

### Save dataframe as csv to drive


In [ ]:
df_res.to_csv('/content/drive/MyDrive/DeepLearning/df_res_squad_coviddata_top5.csv', index=False)